<a href="https://colab.research.google.com/github/falahgs/MaSK-No-Mask-Detection/blob/master/MaSK_No_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mask No Mask Detection 
## By Author 
Falah.G.Saleih


##speicial thanks for pyimageserach blog web site
(https://www.pyimagesearch.com/)

In [0]:
#download dataset 
!wget https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/face-mask-detection/face-mask-detector.zip

--2020-05-07 05:59:22--  https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/face-mask-detection/face-mask-detector.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.244.208
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.244.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74053680 (71M) [application/zip]
Saving to: ‘face-mask-detector.zip’

face-mask-detector. 100%[===================>]  70.62M  28.5MB/s    in 2.5s    

2020-05-07 05:59:24 (28.5 MB/s) - ‘face-mask-detector.zip’ saved [74053680/74053680]



In [0]:
!unzip /content/face-mask-detector.zip

Archive:  /content/face-mask-detector.zip
   creating: face-mask-detector/
   creating: face-mask-detector/dataset/
   creating: face-mask-detector/dataset/with_mask/
  inflating: face-mask-detector/dataset/with_mask/0-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/1-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/10-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/100-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/101-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/103-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/104-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/105-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/106-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/107-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/108-with-mask.jpg  
  inflating: face-mask-detector/dataset/with_mask/109-with-mask.jpg 

In [0]:
!pip install tensorflow==2.0

In [0]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [0]:
dataset='/content/face-mask-detector/dataset'
plot='plot.png'
model_path='/content/face-mask-detector/face_detector/deploy.prototxt'

In [0]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 10
BS = 32

In [0]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)


[INFO] loading images...


In [0]:
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [0]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [0]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [0]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...
Train for 34 steps, validate on 276 samples
Epoch 1/10
34/34 [==============================] - 147s 4s/step - loss: 0.6271 - accuracy: 0.6792 - val_loss: 0.3446 - val_accuracy: 0.8242
Epoch 2/10
34/34 [==============================] - 143s 4s/step - loss: 0.3657 - accuracy: 0.8410 - val_loss: 0.1692 - val_accuracy: 0.9648
Epoch 3/10
34/34 [==============================] - 140s 4s/step - loss: 0.3010 - accuracy: 0.8801 - val_loss: 0.1314 - val_accuracy: 0.9727
Epoch 4/10
34/34 [==============================] - 140s 4s/step - loss: 0.2355 - accuracy: 0.9092 - val_loss: 0.1322 - val_accuracy: 0.9609
Epoch 5/10
34/34 [==============================] - 144s 4s/step - loss: 0.2059 - accuracy: 0.9167 - val_loss: 0.0832 - val_accuracy: 0.9844
Epoch 6/10
34/34 [==============================] - 140s 4s/step - loss: 0.1604 - accuracy: 0.9429 - val_loss: 0.0737 - val_accuracy: 0.9844
Epoch 7/10
34/34 [==============================] - 137s 4s/

In [0]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))


[INFO] evaluating network...
              precision    recall  f1-score   support

   with_mask       0.96      0.99      0.98       138
without_mask       0.99      0.96      0.98       138

    accuracy                           0.98       276
   macro avg       0.98      0.98      0.98       276
weighted avg       0.98      0.98      0.98       276

[INFO] saving mask detector model...


In [0]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)


#save model

In [0]:
# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save('mask-no-mask.h5')

#convert model to tensorflow js 

In [0]:
!pip install tensorflowjs

In [0]:
import tensorflowjs as tfjs
from tensorflow.keras.models import load_model
def keras2tfjs(model_path,dir_out):
	#import tensorflowjs as tfjs
	MODEL_PATH = model_path
	print('Model loading...')
	model=load_model(MODEL_PATH)
	#print('Model loaded. Started serving...')
	tfjs.converters.save_keras_model(model, dir_out)

In [0]:
model_path='/content/mask-no-mask.h5'
dir_out='/content/keras2js'
keras2tfjs(model_path,dir_out)

Model loading...


/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
